# Generating the tables describing excluded candidates

This notebook allows us to fill in the excluded candidate template notes with specific exposure times (for when a candidate has been excluded for reasons related to exposure), then we output properly formatted LaTeX tables for the paper appendix.

## Import Statements

In [1]:
import pandas as pd
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
from copy import copy

from xga.samples.extended import ClusterSample
from xga.products import ExpMap

## Setting up the cosmology to use

As stated in the introduction of the paper, we use a concordance ΛCDM cosmology where Ω$_\rm{M}$=0.3, Ω$_\rm{Λ}$=0.7, and H$_0$=70 km s$^{−1}$Mpc$^{−1}$, consistent with the original eFEDS cluster analysis (and other XCS works)

In [2]:
cosmo = LambdaCDM(70, 0.3, 0.7)

## Setting up paths to FITS images

We define paths to the locations of the 0.5-2.0keV, merged (EPN+EMOS1+EMOS2), exposure map produced by XCS for a given XMM observation - these images are not publically available. 

In [3]:
xmm_ex_path = "/home/dt237/apollo_mnt/xmm_obs/data/{0}/images/{0}-0.50-2.00keVmerged_expmap.fits"

## Loading in the eFEDS-XMM sample

In [4]:
efeds_xmm = pd.read_csv("sample_files/efeds-xmm_sample.csv")
efeds_xmm.head(10)

,ID_SRC,name,RA,DEC,z
0,5702,eFEDS-5702,130.295289,0.866770,0.415210
1,6605,eFEDS-6605,130.352971,0.776778,0.407500
2,1644,eFEDS-1644,130.395834,1.030562,0.506660
3,3008,eFEDS-3008,130.450684,0.819822,0.078375
4,3334,eFEDS-3334,130.507566,0.994977,0.086645
5,144,eFEDS-144,131.369500,3.460887,0.334325
6,7831,eFEDS-7831,132.272302,2.243272,0.395420
7,8602,eFEDS-8602,132.592720,0.268673,0.195760
8,1023,eFEDS-1023,132.616040,0.250854,0.196655
9,6125,eFEDS-6125,132.627176,0.558439,0.191720


## Loading in the optical counterparts catalogue

This is used to give context on whether excluded candidates appear in the 'optically confirmed' catalogue.

In [5]:
opt_count = pd.read_csv("sample_files/efeds_cluster_candidate_optical_counterparts.csv")
opt_count.head(10)

,Name,ID_SRC,RA_CORR,DEC_CORR,RADEC_ERR_CORR,EXT,EXT_ERR,EXT_LIKE,ML_CTS,ML_CTS_ERR,...,DEC_GDMAP_2BEST_COMB,Alpha,Delta,Beta,Ellip,Centershift,DIST_NEXT_OPT_R500,DISTNEXT_XCLUST_MPC,DISTNEXT_XCLUST_R500,MCMF_NWAY_SECTOR
0,eFEDS J093712.9+031652,38,144.303568,3.281043,4.000000,26.965470,5.968361,19.696564,83.25920,16.002958,...,3.278213,0.057188,0.260447,0.291617,0.599467,0.456086,8.338902,7.388609,9.777047,A
1,eFEDS J083811.9-015935,53,129.549569,-1.992963,1.381066,8.415080,0.671214,163.666980,840.92730,32.609806,...,-2.046818,0.131410,0.231523,0.211128,0.482397,0.258212,5.109306,6.826096,5.099522,A
2,eFEDS J093521.0+023234,82,143.837404,2.542872,2.256955,11.646078,0.921796,175.889470,789.41034,32.895790,...,2.580924,0.149712,0.214348,0.227254,0.441991,0.313691,99.900002,0.914686,0.694703,A
3,eFEDS J092121.2+031726,100,140.338476,3.290604,2.786281,31.717770,1.420627,478.553300,1507.95620,56.981632,...,-99.900000,0.126342,0.216794,0.396560,0.548407,0.253045,99.900002,6.280532,4.680137,A
4,eFEDS J085751.7+031039,108,134.465300,3.177539,5.122587,57.368830,1.696070,742.209900,2941.50220,104.567130,...,-99.900000,0.184382,0.298033,0.315817,0.452866,0.265861,1.704293,2.833925,2.126780,A
5,eFEDS J092647.5+050033,137,141.697954,5.009053,3.680027,34.240242,1.704842,505.366760,1269.64820,53.113010,...,4.984427,0.078323,0.137997,0.315292,0.591055,0.260348,99.900002,0.315917,0.230575,A
6,eFEDS J084528.7+032739,144,131.369500,3.460887,3.187693,28.416616,1.641801,316.698500,942.22090,43.392887,...,3.474332,0.130602,0.166659,0.197001,0.046051,0.254389,99.900002,19.738748,15.345604,B1
7,eFEDS J092002.2+010220,150,140.009027,1.038868,2.707373,14.811505,1.150945,179.593930,693.43646,32.079433,...,-99.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B1
8,eFEDS J090131.2+030057,152,135.379989,3.015696,5.006569,33.493958,1.665559,373.949920,1196.78820,51.578068,...,-99.900000,0.158592,0.320781,0.253769,0.489150,0.310878,0.470425,0.486326,0.427307,B1
9,eFEDS J083651.3+030002,197,129.213825,3.000636,3.505422,28.246906,3.193915,153.496120,747.97363,42.946740,...,3.018203,0.150400,0.389033,0.462623,0.140946,0.269811,99.900002,0.165200,0.158356,B1


## Loading in the template notes

These notes were written during the 'eyeballing' stage, but certain parts (specific exposure times) have been left to fill in.

In [6]:
ex_notes = pd.read_csv("excluded_notes_template.tsv", sep='\t')
ex_notes.head(10)

,efeds_id,xmm_issue,tainted_xray,notes
0,8094,True,False,On the edge of the XMM field of view.
1,7700,True,False,On the edge of the XMM field of view.
2,1797,True,False,On the edge of the XMM field of view (the eROS...
3,11836,True,False,"On the edge of the XMM field of view, and the ..."
4,9877,True,False,Low signal-to-noise XMM image (xxx s exposure ...
5,2757,True,False,Low signal-to-noise XMM image (xxx s exposure ...
6,5858,True,False,Low signal-to-noise XMM image (xxx s exposure ...
7,2074,True,False,Low signal-to-noise XMM image (xxx s exposure ...
8,11837,True,False,Low signal-to-noise XMM image (xxx s exposure ...
9,1644,False,False,X-ray emission appears coincident with two int...


## Setting up a ClusterSample for those candidates that have been excluded

To be clear, we have decided that some of these candidates are **not** galaxy clusters, but this is just a convenient way to grab the exposure times we need.

In [7]:
excluded_cands = efeds_xmm[efeds_xmm['ID_SRC'].isin(ex_notes['efeds_id'])]
excluded_cands.head(10)

,ID_SRC,name,RA,DEC,z
0,5702,eFEDS-5702,130.295289,0.866770,0.415210
2,1644,eFEDS-1644,130.395834,1.030562,0.506660
3,3008,eFEDS-3008,130.450684,0.819822,0.078375
4,3334,eFEDS-3334,130.507566,0.994977,0.086645
7,8602,eFEDS-8602,132.592720,0.268673,0.195760
12,1376,eFEDS-1376,133.229719,-1.627119,0.342740
14,8094,eFEDS-8094,133.643768,-1.677446,0.594875
15,7700,eFEDS-7700,133.668558,-2.158824,0.472500
17,5909,eFEDS-5909,133.829629,-1.721076,0.364895
18,1797,eFEDS-1797,133.875584,-1.109703,0.753862


In [8]:
ra = excluded_cands['RA'].values
dec = excluded_cands['DEC'].values
z = excluded_cands['z'].values
n = excluded_cands['name'].values

excluded_srcs = ClusterSample(ra, dec, z, n, r500=Quantity([300]*len(excluded_cands), 'kpc'), 
                              use_peak=False, back_inn_rad_factor=2, back_out_rad_factor=3, clean_obs=True, 
                              clean_obs_reg='r500', clean_obs_threshold=0.7, load_fits=True, cosmology=cosmo)

Setting up Galaxy Clusters:   0%|          | 0/25 [00:00<?, ?it/s]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:109: UserWarning: There are 1 alternative matches for observation 0822470101, associated with source eFEDS-5702
  warnings.warn("There are {0} alternative matches for observation {1}, associated with "
/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: eFEDS-5702 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   4%|▍         | 1/25 [00:03<01:15,  3.13s/it]/home/dt237/code/PycharmProjects/XGA/xga/sources/general.py:109: UserWarning: There are 1 alternative matches for observation 0822470101, associated with source eFEDS-1644
  warnings.warn("There are {0} alternative matches for observation {1}, associated with "
Setting up

## Retrieving the exposure times

Grab exposure times at the candidate coordinates from the 0.5-2.0keV XCS PN+MOS1+MOS2 exposure maps of the highest SNR observations (which we used for eyeballing), for those candidates that have an 'xxx' in their notes, which I added to indicate exposure information is required.

In [9]:
# We make a copy of the exclusion notes to populate with exposure times where necessary
final_ex_notes = ex_notes.copy()

for row_ind, row in ex_notes.iterrows():
    if 'xxx' in row['notes']:
        src = excluded_srcs['eFEDS-{}'.format(row['efeds_id'])]
        # The SNR observation-instrument rankings for a 300kpc aperture
        ranks = src.snr_ranking(Quantity(300, 'kpc'))
        # Selecting the best observation-instrument
        best = ranks[0][-1]
        xmm_ex = ExpMap(xmm_ex_path.format(best[0]), best[0], 'XMM', '', '', '', Quantity(0.5, 'keV'), 
                        Quantity(2.0, 'keV'))
        ext = xmm_ex.get_exp(src.ra_dec).round(0).astype(int).value
        final_ex_notes.loc[row_ind, 'notes'] = final_ex_notes.loc[row_ind, 'notes'].replace('xxx', str(ext))

## Writing out the LaTeX tables

Here we assemble the simple LaTeX code that makes up the tables describing which candidates were excluded and why, we also make an 'optically confirmed' subset of the optical counterpart catalogue - using the eFEDS Klein et al. definition.

In [10]:
opt_conf = opt_count[opt_count['F_CONT_BEST_COMB'] < 0.3]

In [11]:
line_fmt = "{i}{da} & {r} & {d} & {z} & {n} \\\ \n\\hline\n"

xmm_tab = ""
tainted_xray_tab = ""
other_tab = ""

for row_ind, row in final_ex_notes.iterrows():
    i = row['efeds_id']
    r = efeds_xmm[efeds_xmm['ID_SRC'] == i]['RA'].values[0].round(3)
    d = efeds_xmm[efeds_xmm['ID_SRC'] == i]['DEC'].values[0].round(3)
    z = efeds_xmm[efeds_xmm['ID_SRC'] == i]['z'].values[0].round(3)
    
    # Replace XMM and eROSITA strings with emphasised versions for LaTeX
    n = copy(row['notes'])
    n = n.replace('XMM', "{\\em XMM}")
    n = n.replace('eROSITA', "{\\em eROSITA}")
    
    # If the current candidate is in the 'optically confirmed' sample then we dagger the ID
    if i in opt_conf['ID_SRC'].values:
        da = "$^\\dagger$"
    else:
        da = ""
    
    # Populate the line
    cur_line = line_fmt.format(i=i, r=r, d=d, n=n, da=da, z=z)
    
    # Put the line in the appropriate table string
    if row['xmm_issue']:
        xmm_tab += cur_line
    elif row['tainted_xray']:
        tainted_xray_tab += cur_line
    else:
        other_tab += cur_line


Now we print out the LaTeX code for the three tables:

In [12]:
print(xmm_tab)

8094$^\dagger$ & 133.644 & -1.677 & 0.595 & On the edge of the {\em XMM} field of view. \\ 
\hline
7700 & 133.669 & -2.159 & 0.472 & On the edge of the {\em XMM} field of view. \\ 
\hline
1797$^\dagger$ & 133.876 & -1.11 & 0.754 & On the edge of the {\em XMM} field of view (the {\em eROSITA} image confirms presence of an extended source). \\ 
\hline
11836$^\dagger$ & 135.272 & -1.424 & 0.405 & On the edge of the {\em XMM} field of view, and the {\em XMM} observation is shallow (1023 s exposure at the eFEDS coordinates). The {\em eROSITA} image confirms presence of an extended source. \\ 
\hline
9877$^\dagger$ & 136.04 & 0.642 & 0.311 & Low signal-to-noise {\em XMM} image (7489 s exposure at the eFEDS coordinates) which has been affected by flaring.  \\ 
\hline
2757$^\dagger$ & 134.756 & 1.114 & 0.162 & Low signal-to-noise {\em XMM} image (6870 s exposure at the eFEDS coordinates). \\ 
\hline
5858 & 136.687 & 1.19 & 0.441 & Low signal-to-noise {\em XMM} image (15020 s exposure at the eF

In [13]:
print(tainted_xray_tab)

16370$^\dagger$ & 134.098 & -1.604 & 0.425 & eFEDS candidate coincident with a collection of galaxies in SDSS/HSC; however X-ray emission is contaminated by low redshift spiral galaxy. \\ 
\hline
150$^\dagger$ & 140.009 & 1.039 & 0.017 & X-ray extended emission dominated by a massive elliptical galaxy. \\ 
\hline
3133$^\dagger$ & 140.649 & -0.412 & 0.055 & SDSS/HSC indicates the presence of a group of galaxies, however, the eFEDS emission is likely dominated by the central massive elliptical galaxy. \\ 
\hline
3008$^\dagger$ & 130.451 & 0.82 & 0.078 & SDSS/HSC indicates the presence of a group of galaxies, however, the eFEDS emission is likely dominated by the central massive elliptical galaxy. \\ 
\hline



In [14]:
print(other_tab)

1644$^\dagger$ & 130.396 & 1.031 & 0.507 & X-ray emission appears coincident with two interacting active galaxies (identified from {\em XMM} proposal). \\ 
\hline
3334$^\dagger$ & 130.508 & 0.995 & 0.087 & No significant X-ray emission in {\em XMM} or {\em eROSITA}, especially considering low redshift assigned in the eFEDS catalogue. \\ 
\hline
8602$^\dagger$ & 132.593 & 0.269 & 0.196 & This candidate is a cluster that has been split in two, the other part has the eFEDS source ID 1023. \\ 
\hline
5909$^\dagger$ & 133.83 & -1.721 & 0.365 & eFEDS candidate detected as a point source in XCS. There are no galaxies coincident with the X-ray emission in SDSS. \\ 
\hline
8922$^\dagger$ & 134.067 & -1.663 & 0.514 & Spurious detection in the outskirts of an X-ray bright spiral galaxy. \\ 
\hline
9463 & 136.753 & 1.176 & 0.799 & eFEDS candidate detected as two separate point sources in XCS with deeper {\em XMM} (18501 s at eFEDS coordinates) data. \\ 
\hline
13484 & 136.766 & 1.132 & 0.307 & No 

## Saving the formatted exclusion notes

Saving the formatted excluded notes to a new file: 

In [15]:
final_ex_notes.to_csv("excluded_notes.tsv", index=False, sep='\t')